In [394]:
import pandas as pd
import numpy as np

In [395]:
df_train = pd.read_csv('train.csv')

In [396]:
df_test = pd.read_csv('test.csv')

In [397]:
df_train.shape

(1460, 81)

In [398]:
numerical = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 
             'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
             'TotalBsmtSF', '1stFlrSF', '2ndFlrSF','LowQualFinSF', 
             'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
             'HalfBath', 'TotRmsAbvGrd', 'Fireplaces', 'KitchenAbvGr',
             'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 
             'BedroomAbvGr', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 
             'ScreenPorch', 'PoolArea', 'MiscVal']
ordinal = ['OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 
           'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
           'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 
           'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 
           'PavedDrive', 'PoolQC', 'Fence']
binary = ['CentralAir']
cyclical = ['MoSold', 'YrSold']
ids = ['Id'] 
goal = ['SalePrice']
categorical = set(df_train.columns).difference(set(numerical),
                                              set(ordinal), set(binary),
                                              set(cyclical), set(ids),
                                              set(goal))


In [399]:
y_train = df_train.SalePrice
df_full = pd.concat([df_train.drop('SalePrice', axis=1), df_test])
df_full.set_index('Id', inplace=True, drop=True)

In [400]:
################# DEALING WITH NUMERICAL VARIABLES ###########################

In [401]:
# Transform Month and Year sold into a single
df_full.MoSold = (df_full.MoSold - 1)/12
df_full['TimeSold'] = df_full.MoSold + df_full.YrSold
numerical.append('TimeSold')
df_full.drop(cyclical, axis=1, inplace=True)

In [402]:
df_full[numerical].head()

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,GarageArea,WoodDeckSF,BedroomAbvGr,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,TimeSold
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450,2003,2003,196.0,706.0,0.0,150.0,856.0,856,...,548.0,0,3,61,0,0,0,0,0,2008.083333
2,80.0,9600,1976,1976,0.0,978.0,0.0,284.0,1262.0,1262,...,460.0,298,3,0,0,0,0,0,0,2007.333333
3,68.0,11250,2001,2002,162.0,486.0,0.0,434.0,920.0,920,...,608.0,0,3,42,0,0,0,0,0,2008.666667
4,60.0,9550,1915,1970,0.0,216.0,0.0,540.0,756.0,961,...,642.0,0,3,35,272,0,0,0,0,2006.083333
5,84.0,14260,2000,2000,350.0,655.0,0.0,490.0,1145.0,1145,...,836.0,192,4,84,0,0,0,0,0,2008.916667


In [403]:
df_full[numerical].isna().sum()[df_full[numerical].isna().any()]

LotFrontage     486
MasVnrArea       23
BsmtFinSF1        1
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
BsmtFullBath      2
BsmtHalfBath      2
GarageYrBlt     159
GarageCars        1
GarageArea        1
dtype: int64

In [404]:
df_full[numerical] =  df_full[numerical].fillna(0)
# These NAs are related to missing pieces in the house (no garage, no front)
# etc. 0 is adequate as it can be regressed on for areas, while it allows an
# easy split and separation between HAS - HAS NOT

In [405]:
################# DEALING WITH ORDINAL VARIABLES ###########################

In [406]:
ordinal

['OverallQual',
 'OverallCond',
 'ExterQual',
 'ExterCond',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'HeatingQC',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence']

In [407]:
df_full.ExterQual.unique()

array(['Gd', 'TA', 'Ex', 'Fa'], dtype=object)

In [408]:
ordinal_map_1 = {'Ex':5,
                 'Gd':4,
                 'TA':3,
                 'Fa':2,
                 'Po':1,
                 np.nan: 0}
ordinal_map_2 = {'Gd':4,
                 'Av':3,
                 'Mn':2,
                 'No':1,
                 np.nan: 0}
ordinal_map_3 = {'GLQ':6,
                 'ALQ':5,
                 'BLQ':4,
                 'Rec':3,
                 'LwQ':2,
                 'Unf':1,
                 np.nan: 0}
ordinal_map_4 = {'Typ':8,
                 'Min1':7,
                 'Min2':6,
                 'Mod':5,
                 'Maj1':4,
                 'Maj2':3,
                 'Sev':2,
                 'Sav':1}
ordinal_map_5 = {'Fin':3,
                 'RFn':2,
                 'Unf':1,
                 np.nan: 0}
ordinal_map_6 = {'Y':2,
                 'P':2,
                 'N':1,
                 np.nan: 0}
ordinal_map_7 = {'GdPrv':4,
                 'MnPrv':3,
                 'GdWo':2,
                 'MnWw':1,
                 np.nan: 0}

In [409]:
ordinal_2 = ['BsmtExposure']
ordinal_3 = ['BsmtFinType1', 'BsmtFinType2']
ordinal_4 = ['Functional']
ordinal_5 = ['GarageFinish']
ordinal_6 = ['PavedDrive']
ordinal_7 = ['Fence']
ordinal_num = ['OverallCond', 'OverallQual']
ordinal_1 = list(set(ordinal).difference(set(ordinal_2), set(ordinal_3),
                                        set(ordinal_3), set(ordinal_4),
                                        set(ordinal_5), set(ordinal_6),
                                        set(ordinal_7), set(ordinal_num)))

In [410]:
# Kithen and apartments must have quality ratings. Therefore, we will fill
# KitchenQual and Functional missing values with the respective modes.

df_full.loc[df_full[df_full.Functional.isna()].index, 'Functional'] = df_train.Functional.mode()[0]
df_full.loc[df_full[df_full.KitchenQual.isna()].index, 'KitchenQual'] = df_train.KitchenQual.mode()[0]


In [411]:
for elem in ordinal_1:
    print(elem)
    df_full[elem] = df_full[elem].apply(lambda x: ordinal_map_1[x])
for elem in ordinal_2:
    df_full[elem] = df_full[elem].apply(lambda x: ordinal_map_2[x])
for elem in ordinal_3:
    df_full[elem] = df_full[elem].apply(lambda x: ordinal_map_3[x])
for elem in ordinal_4:
    df_full[elem] = df_full[elem].apply(lambda x: ordinal_map_4[x])
for elem in ordinal_5:
    df_full[elem] = df_full[elem].apply(lambda x: ordinal_map_5[x])
for elem in ordinal_6:
    df_full[elem] = df_full[elem].apply(lambda x: ordinal_map_6[x])
for elem in ordinal_7:
    df_full[elem] = df_full[elem].apply(lambda x: ordinal_map_7[x])

HeatingQC
PoolQC
ExterQual
ExterCond
KitchenQual
GarageQual
BsmtQual
FireplaceQu
GarageCond
BsmtCond


In [412]:
################# DEALING WITH CATEGORICAL VARIABLES ###########################

In [413]:
def get_target_value(elem):
    values = df_train[elem].unique()
    return df_train[[elem, 'SalePrice']].groupby(elem).median()
    # Should we return mean or median? We can check this through crossval

In [414]:
df_full.MSSubClass = df_full.MSSubClass.apply(lambda x: str(x))

In [415]:
df_full_ohe = pd.get_dummies(df_full[categorical])

In [416]:
df_full_ohe = df_full_ohe.join(df_full[numerical]).join(df_full[ordinal])

In [417]:
df_train_ohe = df_full_ohe.loc[:1460]
df_train_ohe = df_train_ohe.reset_index(drop=True).join(y_train)
df_test_ohe = df_full_ohe.loc[1460:]

In [419]:
df_train_ohe.to_csv('train_ohe.csv')
df_test_ohe.to_csv('test_ohe.csv')